In [ ]:
from collections import defaultdict

import numpy as np
import matplotlib.pylab as plt
import pandas as pd

# scikit-hep
import awkward as ak
import uproot
import vector
vector.register_awkward()
import correctionlib

### Load data from NTuples without materialising in memory

!! Install `xrootd` via `conda-forge`

In [ ]:
datasets = {
    "signal": "root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/ZprimeToTT_M2000_W20_TuneCP2_PSweights_13TeV-madgraph-pythiaMLM-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/270000/22BAB5D2-9E3F-E440-AB30-AE6DBFDF6C83.root",
    "tt_semilep": "root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/120000/08FCB2ED-176B-064B-85AB-37B898773B98.root",
    "tt_had": "root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/130000/009086DB-1E42-7545-9A35-1433EC89D04B.root",
    "tt_lep": "root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/120000/4A9AFE65-CC58-044A-AFBB-F4CAEA0A7FCD.root",
    "Wjets": "root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/270000/00702195-E707-3743-8BBA-57EB9DEE1DBA.root",
    "data": "root://eospublic.cern.ch//eos/opendata/cms/Run2016H/SingleMuon/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/130000/0DEE1709-0416-F24B-ACB2-C68997CB6465.root"
}

events = defaultdict(dict)
for dataset, filename in datasets.items():
    print(f"Opening {dataset} file: {filename}")
    # Open the file
    with uproot.open(filename) as f:
        f = uproot.open(filename)
        events[dataset]= {"events": f['Events']}
        nevents = events[dataset]["events"].num_entries
        events[dataset].update({"nevents": nevents})
        print(f"{dataset}: {nevents = }")

### Read branches into memory 

In [ ]:
for dataset, data in events.items():
    dataset_events = data["events"]
    print(f"Processing {dataset} dataset")
    # Load the branches we need
    events[dataset]["muon_pt"] = dataset_events["Muon_pt"].array()
    events[dataset]["muon_eta"] = dataset_events["Muon_eta"].array()
    events[dataset]["muon_phi"] = dataset_events["Muon_phi"].array()
    events[dataset]["muon_mass"] = dataset_events["Muon_mass"].array()
    events[dataset]["muon_iso"] = dataset_events["Muon_miniIsoId"].array()
    events[dataset]["muon_tightId"] = dataset_events["Muon_tightId"].array()

    events[dataset]["fatjet_tag"] = dataset_events["FatJet_particleNet_TvsQCD"].array()
    events[dataset]["fatjet_pt"] = dataset_events["FatJet_pt"].array()
    events[dataset]["fatjet_eta"] = dataset_events["FatJet_eta"].array()
    events[dataset]["fatjet_phi"] = dataset_events["FatJet_phi"].array()
    events[dataset]["fatjet_mass"] = dataset_events["FatJet_mass"].array()

    events[dataset]["jet_btag"] = dataset_events["Jet_btagDeepB"].array()
    events[dataset]["jet_jetid"] = dataset_events["Jet_jetId"].array()
    events[dataset]["jet_pt"] = dataset_events["Jet_pt"].array()
    events[dataset]["jet_eta"] = dataset_events["Jet_eta"].array()
    events[dataset]["jet_phi"] = dataset_events["Jet_phi"].array()
    events[dataset]["jet_mass"] = dataset_events["Jet_mass"].array()

    events[dataset]["met_pt"] = dataset_events["PuppiMET_pt"].array()
    events[dataset]["met_eta"] = 0 * dataset_events["PuppiMET_pt"].array()  # Fix this to be 0 !!
    events[dataset]["met_phi"] = dataset_events["PuppiMET_phi"].array()
    #==================== Event variables =========================
    events[dataset]["ht_lep"] = events[dataset]["muon_pt"] + events[dataset]["met_pt"]

    # meta
    if dataset != "data":
        events[dataset]["weights"] = dataset_events["genWeight"].array()
        events[dataset]["nTrueInt"] = dataset_events["Pileup_nTrueInt"].array()
    else:
        events[dataset]["weights"] = np.ones(events[dataset]["nevents"])
        events[dataset]["nTrueInt"] = np.ones(events[dataset]["nevents"])

### Define object and event selection

In [ ]:
#https://github.com/cms-opendata-workshop/workshop2024-lesson-event-selection/blob/main/instructors/dpoa_workshop_utilities.py
def build_lumi_mask(lumifile, tree, verbose=False):

    # lumifile should be the name/path of the file
    good_luminosity_sections = ak.from_json(open(lumifile, 'rb'))
    # Pull out the good runs as integers
    good_runs = np.array(good_luminosity_sections.fields).astype(int)

    # Get the good blocks as an awkward array
    # First loop over to get them as a list
    all_good_blocks = []
    for field in good_luminosity_sections.fields:
        all_good_blocks.append(good_luminosity_sections[field])

    # Turn the list into an awkward array
    all_good_blocks = ak.Array(all_good_blocks)

    # Get the runs and luminosity blocks from the tree
    run = tree['run'].array()
    lumiBlock = tree['luminosityBlock'].array()


    # ChatGPT helped me with this part!
    # Find index of values in arr2 if those values appear in arr1
    def find_indices(arr1, arr2):
        arr1_np = np.asarray(ak.to_numpy(arr1))
        arr2_np = np.asarray(ak.to_numpy(arr2))

        # Sort arr1 and track original indices
        sorter = np.argsort(arr1_np)
        sorted_arr1 = arr1_np[sorter]

        # Search positions
        pos = np.searchsorted(sorted_arr1, arr2_np)

        # Check if arr2 values actually exist in arr1
        valid = (pos < len(arr1_np)) & (sorted_arr1[pos] == arr2_np)

        # Prepare result
        out = np.full(len(arr2_np), -1, dtype=int)
        out[valid] = sorter[pos[valid]]
        return ak.Array(out)

    # Get the indices that say where the good runs are in the lumi file
    # for the runs that appear in the tree
    good_runs_indices = find_indices(good_runs, run)

    # For each event, calculate the difference between the luminosity block for that event
    # and the good luminosity blocks for that run for that event
    diff = lumiBlock - all_good_blocks[good_runs_indices]


    # If the lumi block appears between any of those good block numbers,
    # then one difference will be positive and the other will be negative
    #
    # If it it outside of the range, both differences will be positive or
    # both negative.
    #
    # The product will be negagive if the lumi block is in the range
    # and positive if it is not in the range
    prod_diff = ak.prod(diff, axis=2)
    mask = ak.any(prod_diff<=0, axis=1)

    return mask

In [ ]:
#tau32 = fatjet_tau3/fatjet_tau2
#cut_fatjet = (tau32>0.67) & (fatjet_eta>-2.4) & (fatjet_eta<2.4) & (fatjet_mSD>105) & (fatjet_mSD<220)
for dataset, data in events.items():

    events[dataset]["obj_sel"] = {
        "fatjets": (data["fatjet_pt"] > 500) & (data["fatjet_tag"] > 0.5),
        "muons": (data["muon_pt"] > 55) & (data["muon_eta"] > -2.4) & (data["muon_eta"] < 2.4) &
                (data["muon_tightId"] == True) & (data["muon_iso"] > 1) & (data["ht_lep"] > 150),
        "jets": (data["jet_btag"] > 0.5) & (data["jet_jetid"] >= 4)
    }
    # Event cut
    events[dataset]["event_sel"] = {
        "met": (data["met_pt"] > 50),
        "nmuons": (ak.sum(data["obj_sel"]["muons"], axis=1) == 1),
        "trigger": (data["events"]["HLT_TkMu50"].array()),
        "btag": (ak.sum(data["obj_sel"]["jets"], axis=1) > 0),
        "ntop": (ak.sum(data["obj_sel"]["fatjets"], axis=1) == 1)
    }

    full_event_sel =  ((events[dataset]["event_sel"]["trigger"]) &
        (events[dataset]["event_sel"]["nmuons"]) &
        (events[dataset]["event_sel"]["met"]) &
        (events[dataset]["event_sel"]["ntop"]) &
        (events[dataset]["event_sel"]["btag"]))

    if dataset == "data":
        mask_lumi = build_lumi_mask('Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt', events[dataset]["events"], verbose=False)
        full_event_sel = full_event_sel & (mask_lumi)

    events[dataset]["event_sel"]["full"] = full_event_sel

    full_event_sel = events[dataset]["event_sel"]["full"]

    print(f"Fraction of {dataset} events left: ", ak.sum(events[dataset]["event_sel"]["full"])/nevents, "Total left: ",  ak.sum(events[dataset]["event_sel"]["full"]))

    fatjets = ak.zip(
        {
            "pt": data["fatjet_pt"][full_event_sel][data["obj_sel"]["fatjets"][full_event_sel]],
            "eta": data["fatjet_eta"][full_event_sel][data["obj_sel"]["fatjets"][full_event_sel]],
            "phi": data["fatjet_phi"][full_event_sel][data["obj_sel"]["fatjets"][full_event_sel]],
            "mass": data["fatjet_mass"][full_event_sel][data["obj_sel"]["fatjets"][full_event_sel]],
        },
        with_name="Momentum4D",
    )
    events[dataset]["fatjets"] = fatjets

    muons = ak.zip(
        {
            "pt": data["muon_pt"][full_event_sel][data["obj_sel"]["muons"][full_event_sel]],
            "eta": data["muon_eta"][full_event_sel][data["obj_sel"]["muons"][full_event_sel]],
            "phi": data["muon_phi"][full_event_sel][data["obj_sel"]["muons"][full_event_sel]],
            "mass": data["muon_mass"][full_event_sel][data["obj_sel"]["muons"][full_event_sel]],
        },
        with_name="Momentum4D",
    )
    print(muons)
    events[dataset]["muons"] = muons

    jets = ak.zip(
        {
            "pt": data["jet_pt"][full_event_sel][data["obj_sel"]["jets"][full_event_sel]][:, 0],
            "eta": data["jet_eta"][full_event_sel][data["obj_sel"]["jets"][full_event_sel]][:, 0],
            "phi": data["jet_phi"][full_event_sel][data["obj_sel"]["jets"][full_event_sel]][:, 0],
            "mass": data["jet_mass"][full_event_sel][data["obj_sel"]["jets"][full_event_sel]][:, 0],
        },
        with_name="Momentum4D",
    )
    events[dataset]["jets"] = jets

    met = ak.zip(
        {
            "pt": data["met_pt"][data["event_sel"]["full"]],
            "eta": data["met_eta"][data["event_sel"]["full"]],
            "phi": data["met_phi"][data["event_sel"]["full"]],
            "mass": 0,
        },
        with_name="Momentum4D",
    )
    events[dataset]["met"] = met

    p4mu,p4fj,p4j,p4met = ak.unzip(ak.cartesian([muons, fatjets, jets, met]))
    p4tot = p4mu + p4fj + p4j + p4met
    events[dataset]["p4tot"] = p4tot
    events[dataset]['mtt'] = ak.flatten(p4tot.mass)
    events[dataset]['mu_pt'] = ak.flatten(p4mu.pt)
    events[dataset]['mu_abseta'] = np.abs(ak.flatten(p4mu.eta))
    events[dataset]['weight'] = data["weights"][full_event_sel]
    events[dataset]['pileup'] = data["nTrueInt"][full_event_sel]


In [ ]:
for dataset, data in events.items():
    print(f"Plotting {dataset} dataset")
    p4tot = data["p4tot"]
    # Plotting
    plt.figure(figsize=(10, 7))
    plt.title(f"{dataset} - Fatjet mass")
    plt.hist(ak.flatten(p4tot.mass),bins=50, range=(0,7000));
    plt.xlabel("Fatjet mass [GeV]")
    plt.ylabel("Number of events")
    plt.show()

In [ ]:

for dataset, data in events.items():
    to_write = {}
    to_write['mtt'] = data['mtt']
    to_write['mu_pt'] = data['mu_pt']
    to_write['mu_abseta'] = data['mu_abseta']

    to_write['weight'] = data["weight"]
    to_write['pileup'] = data["pileup"]
    df = pd.DataFrame.from_dict(to_write)
    outfilename = f"output_{dataset}_{filename.split('/')[-1].split('.')[0]}.csv"
    df.to_csv(outfilename, index=False)

In [ ]:
import gzip
with gzip.open("puWeights.json.gz",'rt') as file:
    data = file.read().strip()
    pu_corr_evaluator = correctionlib._core.CorrectionSet.from_string(data)

with gzip.open("./muon_Z.json.gz",'rt') as file:
    data = file.read().strip()
    muo_corr_evaluator = correctionlib._core.CorrectionSet.from_string(data)

In [ ]:
datadict = {}
datadict['signal'] = np.genfromtxt('output_signal_22BAB5D2-9E3F-E440-AB30-AE6DBFDF6C83.csv', delimiter=',', names=True, dtype=float)
datadict['tt_semilep'] = np.genfromtxt('output_tt_semilep_08FCB2ED-176B-064B-85AB-37B898773B98.csv', delimiter=',', names=True, dtype=float)
datadict['tt_had'] = np.genfromtxt('output_tt_had_009086DB-1E42-7545-9A35-1433EC89D04B.csv', delimiter=',', names=True, dtype=float)
datadict['data'] = np.genfromtxt('output_data_0DEE1709-0416-F24B-ACB2-C68997CB6465.csv', delimiter=',', names=True, dtype=float)

hists = {}
for sample in datadict.keys():
    print(sample)
    hists[sample] = {
        "pu": datadict[sample]['pileup'],
        "mu_pt": datadict[sample]['mu_pt'],
        "mu_abseta": datadict[sample]['mu_abseta'],
        "genWeight": datadict[sample]['weight']/np.abs(datadict[sample]['weight']),
        "mtt": datadict[sample]['mtt'],
        "pu_weight": [pu_corr_evaluator["Collisions16_UltraLegacy_goldenJSON"].evaluate(n,"nominal") for n in datadict[sample]['pileup']],
        "pu_weight_up": [pu_corr_evaluator["Collisions16_UltraLegacy_goldenJSON"].evaluate(n,"up") for n in datadict[sample]['pileup']],
        "pu_weight_dn": [pu_corr_evaluator["Collisions16_UltraLegacy_goldenJSON"].evaluate(n,"down") for n in datadict[sample]['pileup']],
        "leadmuon_sf": [muo_corr_evaluator["NUM_TightID_DEN_TrackerMuons"].evaluate(eta,pt,"nominal") for pt,eta in zip(datadict[sample]['mu_pt'],datadict[sample]['mu_abseta'])],
        "leadmuon_sf_up": [muo_corr_evaluator["NUM_TightID_DEN_TrackerMuons"].evaluate(eta,pt,"systup") for pt,eta in zip(datadict[sample]['mu_pt'],datadict[sample]['mu_abseta'])],
        "leadmuon_sf_dn": [muo_corr_evaluator["NUM_TightID_DEN_TrackerMuons"].evaluate(eta,pt,"systdown") for pt,eta in zip(datadict[sample]['mu_pt'],datadict[sample]['mu_abseta'])],
    }

In [ ]:
plt.hist(hists['signal']['mtt'],
         bins=50,range=(0,3000),
         weights=hists['signal']['genWeight']*hists['signal']['pu_weight'],
         histtype="step",color="k",label="nominal")
plt.hist(hists['signal']['mtt'],
         bins=50,range=(0,3000),
         weights=hists['signal']['genWeight']*hists['signal']['pu_weight_up'],
         histtype="step",color="r",label="up")
plt.hist(hists['signal']['mtt'],
         bins=50,range=(0,3000),
         weights=hists['signal']['genWeight']*hists['signal']['pu_weight_dn'],
         histtype="step",color="b",label="down")
plt.legend()
plt.xlabel('ttbar mass (GeV)')
plt.ylabel('events / 60 GeV')
plt.show()

plt.hist(hists['signal']['pu'],
         bins=50,range=(0,50),
         weights=hists['signal']['genWeight']*hists['signal']['pu_weight'],
         histtype="step",color="k",label="nominal")
plt.hist(hists['signal']['pu'],
         bins=50,range=(0,50),
         weights=hists['signal']['genWeight']*hists['signal']['pu_weight_up'],
         histtype="step",color="r",label="up")
plt.hist(hists['signal']['pu'],
         bins=50,range=(0,50),
         weights=hists['signal']['genWeight']*hists['signal']['pu_weight_dn'],
         histtype="step",color="b",label="down")
plt.legend()
plt.xlabel('N true interactions')
plt.ylabel('events')
plt.show()

plt.hist(hists['signal']['mu_pt'],
         bins=50,range=(0,50),
         weights=hists['signal']['genWeight']*hists['signal']['leadmuon_sf'],
         histtype="step",color="k",label="nominal")
plt.hist(hists['signal']['pu'],
         bins=50,range=(0,50),
         weights=hists['signal']['genWeight']*hists['signal']['leadmuon_sf_up'],
         histtype="step",color="r",label="up")
plt.hist(hists['signal']['pu'],
         bins=50,range=(0,50),
         weights=hists['signal']['genWeight']*hists['signal']['leadmuon_sf_dn'],
         histtype="step",color="b",label="down")
plt.legend()
plt.xlabel('Muon pT (GeV)')
plt.ylabel('events')
plt.show()

In [ ]:
import pickle
with open('hists_for_ROOT.p','wb') as f:
    pickle.dump(hists,f)

In [ ]:
import hist

info = {'signal': {"xsec": 1.0, "binning": (0, 3000, 50)},
        'tt_semilep': {"xsec": 831.76*0.438, "binning": (0, 3000, 50)},
        'tt_had': {"xsec": 831.76*0.457, "binning": (0, 3000, 50)},
        }

with open('hists_for_ROOT.p','rb') as f:
    hists = pickle.load(f)


SYSTEMATICS = {
    "leadmuon_sf": {
        "up": "leadmuon_sf_up",
        "dn": "leadmuon_sf_dn",
        "nominal": "leadmuon_sf",
        "type": "weight",
    },
    "pu_weight": {
        "up": "pu_weight_up",
        "dn": "pu_weight_dn",
        "nominal": "pu_weight",
        "type": "weight",
    },
}

observable_name = "mtt"
ndhist = ( hist.Hist.new.Reg(50, 0, 3000, name="observable", label=f"{observable_name} [GeV]")
          .StrCat([], name="process", label="Process", growth=True)
          .StrCat([], name="variation", label="Systematic variation", growth=True)
          .Weight()
        )

for process, process_info in hists.items():

    print(f"Processing {process} dataset")
    # load observable data
    observable = process_info[observable_name]
    # Handle weights
    print(process)
    if process == "data":
        weight = 1.0
    else:
        genweight = process_info['genWeight']
        n_gen = sum(genweight)
        lumi_weight = 16400*info[process]["xsec"]*process_info['genWeight']/n_gen
        pu_weight = process_info['pu_weight']
        mu_id_sf = process_info['leadmuon_sf']
        weight = genweight * lumi_weight * pu_weight * mu_id_sf

    # Fill nominal histogram
    ndhist.fill(
                observable=observable, process=process,
                variation="nominal", weight=weight,
            )
    if process == "data":
            continue
    for variation, variation_info in SYSTEMATICS.items():
        for direction in ["up", "dn"]:

            syst_var_name = f"{variation}_{direction}"
            var_weight = weight
            if variation_info["type"] == "weight":
                var_weight *= np.asarray(process_info[variation_info[direction]])/np.asarray(process_info[variation_info["nominal"]])

            ndhist.fill(
                        observable=observable, process=process,
                        variation=syst_var_name, weight=var_weight,
                    )

print(ndhist)

In [ ]:
import uproot

hists_dict = {"channel": ndhist}
def save_histograms(hist_dict, filename, add_offset=False):
    with uproot.recreate(filename) as f:
        # save all available histograms to disk
        for channel, histogram in hist_dict.items():
            # optionally add minimal offset to avoid completely empty bins
            # (useful for the ML validation variables that would need binning adjustment
            # to avoid those)
            if add_offset:
                histogram += 1e-6
                # reference count for empty histogram with floating point math tolerance
                empty_hist_yield = histogram.axes[0].size*(1e-6)*1.01
            else:
                empty_hist_yield = 0

            for sample in histogram.axes[1]:
                for variation in histogram[:, sample, :].axes[1]:
                    variation_string = "" if variation == "nominal" else f"_{variation}"
                    current_1d_hist = histogram[:, sample, variation]

                    if sum(current_1d_hist.values()) > empty_hist_yield:
                        # only save histograms containing events
                        f[f"{channel}_{sample}{variation_string}"] = current_1d_hist

            # Need to build some notion of pseudodata here

save_histograms(hists_dict, "test_hists.root", add_offset=False)